In [9]:
# LLM Zoomcamp – Homework 1: Introduction
# Author: Pavlo Skorodziievskyi
# Container Startup (Q1)
# --------------------------
# Run this in a notebook shell cell or terminal
!docker run -d \
  --name elasticsearch \
  -p 9200:9200 \
  -e "discovery.type=single-node" \
  -e "xpack.security.enabled=false" \
  docker.elastic.co/elasticsearch/elasticsearch:8.17.6

4e3a404bb47195cb0824dfc8dfe1410ffa967196ff2bcf7986fbd76d580f195e


In [15]:
# helath check
!curl localhost:9200

{
  "name" : "4e3a404bb471",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "i3oSddDwTBacMRRvtwbtNA",
  "version" : {
    "number" : "8.17.6",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",
    "build_date" : "2025-04-30T14:07:12.231372970Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [17]:
!pip uninstall -y elasticsearch elastic-transport urllib3

In [19]:
# Install the library if not already
# !pip install elasticsearch==8.12.1 --force-reinstall --no-cache-dir
!pip install "urllib3<2" "elasticsearch==8.12.1"

  Using cached urllib3-1.26.20-py2.py3-none-any.whl.metadata (50 kB)
  Using cached elasticsearch-8.12.1-py3-none-any.whl.metadata (5.3 kB)
  Using cached elastic_transport-8.17.1-py3-none-any.whl.metadata (3.8 kB)
Using cached elasticsearch-8.12.1-py3-none-any.whl (432 kB)
Using cached urllib3-1.26.20-py2.py3-none-any.whl (144 kB)
Using cached elastic_transport-8.17.1-py3-none-any.whl (64 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.1 requires packaging<25,>=20, but you have packaging 25.0 which is incompatible.
conda-repo-cli 1.0.114 requires urllib3>=2.2.2, but you have urllib3 1.26.20 which is incompatible.


In [21]:
import elasticsearch
print("Elasticsearch client version:", elasticsearch.__version__)
assert elasticsearch.__version__[0] == 8, "Wrong ES client version loaded."

from elasticsearch import Elasticsearch

# Connect to your local ES instance
es = Elasticsearch("http://localhost:9200")

# Get cluster info
cluster_info = es.info()
print("Q1. Build Hash:", cluster_info['version']['build_hash'])

Elasticsearch client version: (8, 12, 1)
Q1. Build Hash: dbcbbbd0bc4924cfeb28929dc05d82d662c527b7


In [39]:
import elasticsearch
print(elasticsearch.__version__)

(9, 0, 2)


In [ ]:
# Q2:

In [23]:
import requests

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

print(f"Total documents loaded: {len(documents)}")
documents[0]

Total documents loaded: 948


{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [25]:
# Create index
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

# Delete the index if it exists (for clean re-runs)
if es.indices.exists(index="faq"):
    es.indices.delete(index="faq")

mapping = {
    "mappings": {
        "properties": {
            "question": {"type": "text"},
            "text": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

es.indices.create(index="faq", body=mapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'faq'})

In [27]:
for doc in documents:
    es.index(index="faq", document=doc)

In [33]:
query = {
    "query": {
        "multi_match": {
            "query": "How do execute a command on a Kubernetes pod?",
            "fields": ["question^4", "text"],
            "type": "best_fields"
        }
    }
}

res = es.search(index="faq", body=query)
top_hit = res['hits']['hits'][0]

print(f"Q3. Top _score: {top_hit['_score']:.2f}")
print("Top Question:", top_hit['_source']['question'])

Q3. Top _score: 44.51
Top Question: How do I debug a docker container?


In [35]:
# Q4
query = {
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": "How do copy a file to a Docker container?",
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    },
    "size": 3
}

res = es.search(index="faq", body=query)

for i, hit in enumerate(res['hits']['hits'], 1):
    print(f"{i}. {hit['_source']['question']}")

1. How do I debug a docker container?
2. How do I copy files from my local machine to docker container?
3. How do I copy files from a different folder into docker container’s working directory?


In [37]:
# Q5:
context_template = """
Q: {question}
A: {text}
""".strip()

context = "\n\n".join([
    context_template.format(
        question=hit["_source"]["question"],
        text=hit["_source"]["text"]
    )
    for hit in res["hits"]["hits"]
])

In [39]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

prompt = prompt_template.format(
    question="How do copy a file to a Docker container?",
    context=context
)


In [41]:
print("Q5. Prompt length:", len(prompt))

Q5. Prompt length: 1446
